# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
#from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(42)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=10  # 批次大，show_step就小点
max_epoch=60  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=256, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=256, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=1, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [3]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
# From https://github.com/Z0m6ie/CIFAR-10_PyTorch
#model = mobilenet(num_classes=10, large_img=False)

# From https://github.com/kuangliu/pytorch-cifar 
if torch.cuda.is_available():
    model=MobileNet(10).cuda()
else:
    model=MobileNet(10)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [5]:
# Implement validation
def train(epoch):
    model.train()
    #writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            if batch_idx % 4*show_step==0:
                print(torch.abs(model.layers[12].conv2.weight).sum())
                print(model.layers[12].conv2.weight.shape)
                print(model.layers[12].conv2.weight)
                
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            #writer.add_scalar('Loss/Loss', loss.item(), epoch)
            #writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [6]:
def validate(epoch):
    model.eval()
    #writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    #writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    #writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [7]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [8]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [ ]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
#writer = SummaryWriter()
#writer.export_scalars_to_json("./all_scalars.json")

#writer.close()

#---------------------------- Test ------------------------------
test(epoch)

tensor(17988.1211, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.4487e-02]],

         [[ 3.5635e-03]],

         [[-4.0969e-02]],

         ...,

         [[ 1.8060e-02]],

         [[ 1.5117e-02]],

         [[-1.2346e-02]]],


        [[[-2.3300e-02]],

         [[ 4.1171e-02]],

         [[ 7.8488e-04]],

         ...,

         [[-7.6127e-03]],

         [[-6.8355e-03]],

         [[-1.8507e-02]]],


        [[[ 1.6536e-02]],

         [[ 3.9632e-03]],

         [[ 1.3682e-02]],

         ...,

         [[ 8.9277e-03]],

         [[ 3.7227e-02]],

         [[-1.3540e-02]]],


        ...,


        [[[-1.7087e-02]],

         [[-3.0224e-02]],

         [[ 1.2771e-02]],

         ...,

         [[-7.3359e-03]],

         [[-4.0879e-03]],

         [[-2.7311e-02]]],


        [[[ 3.2838e-02]],

         [[ 4.7380e-03]],

         [[ 9.6955e-03]],

         ...,

         [[-1.0123e-02]],

         [[ 1.7591e-02]],

         [[-2.9259e-02]]],


 

Train Epoch: 0 [33280/50000 (74%)]	Loss: 1.670058, Accuracy: 35.94
tensor(60073.5156, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 2.3852e-03]],

         [[ 5.3848e-02]],

         [[-4.3827e-02]],

         ...,

         [[-4.1830e-02]],

         [[ 9.2196e-02]],

         [[ 3.9287e-02]]],


        [[[-3.5061e-02]],

         [[ 4.9987e-03]],

         [[-1.1301e-02]],

         ...,

         [[ 2.0078e-02]],

         [[ 4.1882e-02]],

         [[-9.2231e-02]]],


        [[[-4.7300e-02]],

         [[-5.5596e-02]],

         [[-1.0272e-01]],

         ...,

         [[-6.2547e-02]],

         [[ 4.6637e-02]],

         [[ 1.3586e-02]]],


        ...,


        [[[-1.6603e-02]],

         [[-9.3517e-02]],

         [[ 2.9918e-02]],

         ...,

         [[-6.8930e-02]],

         [[-7.6547e-02]],

         [[-5.5144e-02]]],


        [[[ 8.4732e-02]],

         [[-8.2268e-02]],

         [[-2.9441e-02]],

         ...,

         [[-9.711

Train Epoch: 1 [23040/50000 (51%)]	Loss: 1.351730, Accuracy: 51.95
tensor(63600.9062, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[-1.4336e-04]],

         [[ 5.3848e-02]],

         [[-4.5779e-02]],

         ...,

         [[-4.1830e-02]],

         [[ 9.2188e-02]],

         [[ 3.8536e-02]]],


        [[[-1.1115e-01]],

         [[ 4.9987e-03]],

         [[-2.9073e-02]],

         ...,

         [[ 2.0078e-02]],

         [[ 4.1329e-02]],

         [[-9.3384e-02]]],


        [[[-5.2911e-02]],

         [[-5.5596e-02]],

         [[-8.5745e-02]],

         ...,

         [[-6.2547e-02]],

         [[ 4.7853e-02]],

         [[ 1.1906e-02]]],


        ...,


        [[[-1.0457e-02]],

         [[-9.3518e-02]],

         [[ 2.6954e-02]],

         ...,

         [[-6.8930e-02]],

         [[-7.6480e-02]],

         [[-5.4184e-02]]],


        [[[ 8.4278e-02]],

         [[-8.2269e-02]],

         [[ 1.5667e-02]],

         ...,

         [[-9.711

Train Epoch: 2 [12800/50000 (28%)]	Loss: 1.045633, Accuracy: 62.89
tensor(66431.7344, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[-4.8280e-03]],

         [[ 5.3848e-02]],

         [[-5.1918e-02]],

         ...,

         [[-4.1830e-02]],

         [[ 9.2177e-02]],

         [[ 3.6951e-02]]],


        [[[-1.4538e-01]],

         [[ 4.9987e-03]],

         [[-4.5755e-02]],

         ...,

         [[ 2.0078e-02]],

         [[ 4.2177e-02]],

         [[-6.5993e-03]]],


        [[[-6.0469e-02]],

         [[-5.5596e-02]],

         [[-4.9439e-02]],

         ...,

         [[-6.2547e-02]],

         [[ 4.7861e-02]],

         [[-9.5585e-03]]],


        ...,


        [[[-1.0790e-02]],

         [[-9.3518e-02]],

         [[ 2.7388e-02]],

         ...,

         [[-6.8930e-02]],

         [[-7.6475e-02]],

         [[-5.4374e-02]]],


        [[[ 9.4396e-02]],

         [[-8.2269e-02]],

         [[ 7.2087e-03]],

         ...,

         [[-9.711

Train Epoch: 3 [2560/50000 (6%)]	Loss: 0.926071, Accuracy: 66.02
tensor(68263.4141, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[-1.0045e-02]],

         [[ 5.3848e-02]],

         [[-5.1948e-02]],

         ...,

         [[-4.1830e-02]],

         [[ 9.2177e-02]],

         [[ 3.6829e-02]]],


        [[[-1.7431e-01]],

         [[ 4.9987e-03]],

         [[-6.7912e-02]],

         ...,

         [[ 2.0078e-02]],

         [[ 4.2181e-02]],

         [[-6.0098e-03]]],


        [[[-9.5383e-02]],

         [[-5.5596e-02]],

         [[-3.8133e-02]],

         ...,

         [[-6.2547e-02]],

         [[ 4.7862e-02]],

         [[ 6.6307e-03]]],


        ...,


        [[[-9.9885e-03]],

         [[-9.3518e-02]],

         [[ 2.8038e-02]],

         ...,

         [[-6.8930e-02]],

         [[-7.6475e-02]],

         [[-5.4370e-02]]],


        [[[ 1.3315e-01]],

         [[-8.2269e-02]],

         [[ 8.5623e-03]],

         ...,

         [[-9.7114e

Train Epoch: 3 [38400/50000 (85%)]	Loss: 0.863880, Accuracy: 69.92
tensor(69752.4922, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[-1.3681e-02]],

         [[ 5.3848e-02]],

         [[-5.2746e-02]],

         ...,

         [[-4.1830e-02]],

         [[ 9.2177e-02]],

         [[ 3.6875e-02]]],


        [[[-2.0430e-01]],

         [[ 4.9987e-03]],

         [[-1.1776e-01]],

         ...,

         [[ 2.0078e-02]],

         [[ 4.2181e-02]],

         [[-2.2851e-02]]],


        [[[-1.1044e-01]],

         [[-5.5596e-02]],

         [[ 4.9828e-03]],

         ...,

         [[-6.2547e-02]],

         [[ 4.7862e-02]],

         [[-1.1749e-02]]],


        ...,


        [[[-8.2792e-03]],

         [[-9.3518e-02]],

         [[ 2.5115e-02]],

         ...,

         [[-6.8930e-02]],

         [[-7.6475e-02]],

         [[-5.5273e-02]]],


        [[[ 1.3888e-01]],

         [[-8.2269e-02]],

         [[-1.0386e-03]],

         ...,

         [[-9.711

# Step 3: Test

In [12]:
test(epoch)


Test set: Average loss: 0.4542, Accuracy: 8932/10000 (89.32%)

